In [1]:
'''
Example: Unsupervised learning with the K-means algorithm.
The K-means algorithm is an unsupervised learning algorithm. 
The algorithm doesn’t need trainings data for fitting. 
The algorithm is calibrated for two clusters: a match cluster and a non-match cluster. 
'''

'\nExample: Unsupervised learning with the ECM algorithm.\nTrain data is often hard to collect in record linkage or data matching\nproblems. The Expectation-Conditional Maximisation (ECM) algorithm is the most\nwell known algorithm for unsupervised data matching. The algorithm preforms\nrelatively well compared to supervised methods.\n'

In [2]:
from __future__ import print_function

import numpy as np

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [3]:
from recordlinkage.datasets import load_febrl3

'''
This data set contains 5000 records (2000 originals and 3000 duplicates), 
with a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). 
Distribution of duplicates: 
    168 originals records have 5 duplicate records 
    161 originals records have 4 duplicate records 
    212 originals records have 3 duplicate records 
    256 originals records have 2 duplicate records 
    368 originals records have 1 duplicate record 1
    835 originals records have no duplicate record   
'''

'\nThis data set contains 5000 records (2000 originals and 3000 duplicates), \nwith a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). \nDistribution of duplicates: \n    168 originals records have 5 duplicate records \n    161 originals records have 4 duplicate records \n    212 originals records have 3 duplicate records \n    256 originals records have 2 duplicate records \n    368 originals records have 1 duplicate record 1\n    835 originals records have no duplicate record   \n'

In [4]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [5]:
# load dataset
print('Loading data...')
dfA, true_links = load_febrl3(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(true_links), 'links in dataset A')

Loading data...
5000 records in dataset A
6538 links in dataset A


In [6]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA)
print(len(candidate_links), 'candidate links in dataset A')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.43s - pairs: 75984/12497500 - rr: 0.99392
INFO:recordlinkage:indexing [1/?] - time: 0.43s - pairs_total: 75984/12497500 - rr_total: 0.99392
75984 candidate links in dataset A


In [7]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 1.41s - pairs: 75984
INFO:recordlinkage:comparing [1/?] - time: 1.41s - pairs_total: 75984
feature shape (75984, 7)


In [8]:
# Initialise the Expectation-Conditional Maximisation classifier.
cl = rl.KMeansClassifier()
cl.fit(features)

INFO:recordlinkage:Classification - start training KMeansClassifier
INFO:recordlinkage:Classification - training computation time: ~0.06s


In [10]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches
Predicted number of links: 6044


In [11]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[ 6044   494]
 [    0 69446]]


In [12]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9607375615959306
recall 0.9244417252982563
precision 1.0
